# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import f1_score

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "/home/bibagimon/nanodegree/traffic-signs-data/train.p"
validation_file= "/home/bibagimon/nanodegree/traffic-signs-data/valid.p"
testing_file = "/home/bibagimon/nanodegree/traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_validation = X_valid.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

In [ ]:
import csv

class_names = dict()
with open('signnames.csv', 'r') as csvFile:
    reader = csv.reader(csvFile, delimiter=',')
    
    header_skipped=False
    for row in reader:
        if not header_skipped:
            header_skipped=True
            continue
        class_names[int(row[0])]=row[1]
for key,val in class_names.items():
    print(key,val)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
axs = axs.ravel()

for i, dataset in zip(range(3), [("Training", y_train), ("Validation", y_valid), ("Test", y_test)]):
    hist = np.histogram(dataset[1], bins=range(44))
    axs[i].bar(range(43), hist[0], facecolor='g', alpha=0.75)
    axs[i].set_title(dataset[0])

plt.savefig("examples/visualization1.jpg")
plt.show()


### Visualize Data

In [ ]:
images_by_class = dict()

for i, label in zip(range(len(y_train)), y_train):
    if label not in images_by_class.keys():
        images_by_class[label] = []
    images_by_class[label].append(i)

In [ ]:
import random

sample_images = np.squeeze([
    [random.randint(0, len(images_by_class[cls_indx])-1) for _ in range(5)] for cls_indx in range(n_classes)
])

def show_class_sample(images_by_class, num_rows=int(n_classes/3)+1, num_cols=15):
    fig, axs = plt.subplots(num_rows, num_cols, figsize=(int(num_cols*1.3), int(num_rows*1.6)))
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    axs = axs.ravel()

    for r in range(num_rows):
        for c in range(num_cols):
            i = r * num_cols + c
            
            cls_indx = int(i/5)            
            if cls_indx < n_classes:
                img_indx = sample_images[cls_indx][c % 5]                
                image = X_train[images_by_class[cls_indx][img_indx]]
                
                axs[i].imshow(image)
                axs[i].set_title(cls_indx)
            axs[i].axis('off')

    plt.savefig("examples/visualization2.jpg")
    plt.show()
    
show_class_sample(images_by_class)

### Observations
* The training samples contains a large variation in brighness, warping, background, rotation etc.
* However, the dataset is skewed. So the dataset will be augmented specially for the signs that has less samples

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

#### Helper Functions for Image Augmentation

In [ ]:
import cv2
from skimage import exposure
from skimage.transform import rotate
from skimage.util import random_noise
from scipy import ndimage

def rotate_image(image):
    # grab the dimensions of the image and calculate the center
    # of the image
    (h, w) = image.shape[:2]
    center = (w / 2, h / 2)
    
    angle = np.random.randint(-20.0,20.0)    
    M = cv2.getRotationMatrix2D(center, angle, 1)
    rotated = cv2.warpAffine(image, M, (w, h))
    
    return rotated

def scale_image(image):
    (h, w) = image.shape[:2]
    
    px = np.random.randint(-2,6)
    
    src = np.float32([[px, px],[w-px, px],[px, h-px],[w-px,h-px]])
    
    dst = np.float32([[0,0], [w,0],[0, h],[w,h]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    scaled = cv2.warpPerspective(image, M, (w, h))

    return scaled

def equalize_hist_color(image):
    ycrcb = cv2.cvtColor(image, cv2.COLOR_RGB2YCR_CB)
    channels = cv2.split(ycrcb)

    cv2.equalizeHist(channels[0], channels[0])
    cv2.merge(channels, ycrcb)    

    return cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2RGB)    
    
def rescale_intensity(image):
    v_min, v_max = np.percentile(image, (2, 98))
    
    return exposure.rescale_intensity(image, in_range=(v_min, v_max))

def adjust_gamma(image):
    return exposure.adjust_gamma(image, gamma=0.4, gain=0.9)

In [ ]:
image = X_train[11102]

transform_exmpl = [image,
                   rotate_image(image),
                   scale_image(image),
                   equalize_hist_color(image),
                   rescale_intensity(image),
                   adjust_gamma(image)
                  ]

transform_descs = ['Original', 'Rotate', 'Scale', 'Equalization', "Rescale Intensity", "Adjust Gamma"]
n_cells = len(transform_descs)

f, cells = plt.subplots(1, n_cells, figsize=(16,1))
for i in range(n_cells):
    cells[i].imshow(transform_exmpl[i])
    cells[i].axis('off')
    cells[i].set_title(transform_descs[i])

In [ ]:
def flip_x_axis(image):
    return image[::-1, :, :]

def flip_y_axis(image):
    return image[:, ::-1, :]

def show_flip_sample(flip):
    fig, axs = plt.subplots(2, len(flip), figsize=(len(flip), 3))
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    axs = axs.ravel()

    for c in range(len(flip)):
        axs[c].axis('off')
        axs[c].set_title(flip[c][0])
        axs[c].imshow(X_train[images_by_class[flip[c][0]][100]])

        i = len(flip) + c
        axs[i].axis('off')
        axs[i].set_title(flip[c][1])
        axs[i].imshow(flip_y_axis(X_train[images_by_class[flip[c][0]][100]]))
    plt.show()

In [ ]:
_y_flip = [
    (11, 11), (12, 12), (13, 13), (15, 15),
    (18, 18),(19, 20), (20, 19), (26, 26), (33, 34),
    (34, 33), (36, 37), (37, 36), (38, 39), (39, 38)
]

show_flip_sample(_y_flip)

#### Augment training data

In [ ]:
X_train_extended = []
y_train_extended = []

flipped_images = []
flipped_labels = []

scaled_images = []
scaled_labels = []

rotated_images = []
rotated_labels = []

intensity_adj_images = []
intensity_adj_labels = []

hist_eq_images = []
hist_eq_labels = []

gamma_adj_images = []
gamma_adj_labels = []

hist, bins = np.histogram(y_train, bins=range(44))

y_flip = dict()
for c, _c in _y_flip: y_flip[c] = _c
    
import random as rnd
rnd.seed(9001)

for i, data in zip(range(len(y_train)), y_train):    
    if data in y_flip.keys():
        _c = y_flip[data]

        flipped_images.append(flip_y_axis(X_train[i]))
        flipped_labels.append(_c)
        hist[data] += 1

    if hist[data] < 600:
        scaled_images.append(scale_image(X_train[i]))
        scaled_labels.append(data)
        
        rotated_images.append(rotate_image(X_train[i]))
        rotated_labels.append(data)
        
        intensity_adj_images.append(rescale_intensity(X_train[i]))
        intensity_adj_labels.append(data)
        
        hist_eq_images.append(equalize_hist_color(X_train[i]))
        hist_eq_labels.append(data)
        
        gamma_adj_images.append(adjust_gamma(X_train[i]))
        gamma_adj_labels.append(data)
        
        hist[data] += 5
    elif rnd.randint(1, 100) > 50:
        intensity_adj_images.append(rescale_intensity(X_train[i]))
        intensity_adj_labels.append(data)
        
        hist_eq_images.append(equalize_hist_color(X_train[i]))
        hist_eq_labels.append(data)
        
        gamma_adj_images.append(adjust_gamma(X_train[i]))
        gamma_adj_labels.append(data)
        
        hist[data] += 3

In [ ]:
X_train_augmented = np.concatenate((X_train, 
                                    np.array(flipped_images),
                                    np.array(scaled_images),
                                    np.array(rotated_images),
                                    np.array(intensity_adj_images),
                                    np.array(hist_eq_images),
                                    np.array(gamma_adj_images)))

y_train_augmented = np.concatenate((y_train, 
                                    np.array(flipped_labels),
                                    np.array(scaled_labels),
                                    np.array(rotated_labels),
                                    np.array(intensity_adj_labels),
                                    np.array(hist_eq_labels),
                                    np.array(gamma_adj_labels)))
                                  

fig, axs = plt.subplots(1, 2, figsize=(20, 5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
axs = axs.ravel()

for i, dataset in zip(range(2), [("Training (Before extension)", y_train), ("Training (After extension)", y_train_augmented)]):
    hist, bins = np.histogram(dataset[1], bins=range(44))
    axs[i].bar(range(43), hist, facecolor='g', alpha=0.75)
    axs[i].set_title(dataset[0])
plt.show()

### Preprocessing

In [ ]:
def grayscale(dataset):
    return np.sum(dataset/3, axis=3, keepdims=True)

X_train_gray = grayscale(X_train_augmented)

X_valid_gray = grayscale(X_valid)
X_test_gray = grayscale(X_test)

## Implement LeNet-5
Implement the [LeNet-5](http://yann.lecun.com/exdb/lenet/) neural network architecture.


![LeNet Architecture](lenet.png)
Source: Yan LeCun

### Input
The LeNet architecture accepts a 32x32xC image as input, where C is the number of color channels. Since traffic signal images are converted to grayscale, C is 1 in this case.

### Architecture
**Layer 1: Convolutional.** The output shape should be 28x28x6.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 14x14x6.

**Layer 2: Convolutional.** The output shape should be 10x10x16.

**Activation.** Your choice of activation function.

**Pooling.** The output shape should be 5x5x16.

**Flatten.** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D. The easiest way to do is by using `tf.contrib.layers.flatten`, which is already imported for you.

**Layer 3: Fully Connected.** This should have 120 outputs.

**Activation.** Your choice of activation function.

**Layer 4: Fully Connected.** This should have 84 outputs.

**Activation.** Your choice of activation function.

**Layer 5: Fully Connected (Logits).** This should have 43 outputs.

### Output
Return the result of the 2nd fully connected layer.

In [ ]:
class Conv:
    def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID', mu=0, sigma=0.1):
        # To support reproducability
        tf.set_random_seed(9001)
        
        self.W = tf.Variable(tf.truncated_normal(shape=filter_shape, mean=mu, stddev=sigma))
        self.b = tf.Variable(tf.zeros(filter_shape[3])) # バイアスはフィルタごと
        self.function = function
        self.strides = strides
        self.padding = padding

    def f_prop(self, x):
        u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
        return self.function(u)
    
class Pooling:
    def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
        self.ksize = ksize
        self.strides = strides
        self.padding = padding
    
    def f_prop(self, x):
        return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
class Flatten:
    def f_prop(self, x):
        return tf.contrib.layers.flatten(x)
    
class Dense:
    def __init__(self, in_dim, out_dim, function=lambda x: x, mu=0, sigma=0.1):
        self.W = tf.Variable(tf.truncated_normal(shape=(in_dim, out_dim), mean=mu, stddev=sigma))
        
        self.b = tf.Variable(tf.zeros(out_dim))
        self.function = function

    def f_prop(self, x):
        return self.function(tf.matmul(x, self.W) + self.b)
    
class Activation:
    def __init__(self, function=lambda x: x):
        self.function = function
    
    def f_prop(self, x):
        return self.function(x)
    
class Dropout:
    def __init__(self, keep_prob=0.5):
        self.keep_prob = keep_prob
    
    def f_prop(self, x):
        return tf.nn.dropout(x, keep_prob=self.keep_prob)
    
class BatchNorm:
    def __init__(self, shape, epsilon=np.float32(1e-5)):
        self.gamma = tf.Variable(np.ones(shape, dtype='float32'))
        self.beta  = tf.Variable(np.zeros(shape, dtype='float32'))
        self.epsilon = epsilon

    def f_prop(self, x):
        if len(x.get_shape()) == 2:
            mean, var = tf.nn.moments(x, axes=0, keepdims=True)
            std = tf.sqrt(var + self.epsilon)
        elif len(x.get_shape()) == 4:
            mean, var = tf.nn.moments(x, axes=(0,1,2), keep_dims=True)
            std = tf.sqrt(var + self.epsilon)
        normalized_x = (x - mean) / std
        return self.gamma * normalized_x + self.beta

class LeNet:
    def __init__(self, n_channel=3, use_batch_normalization=False):
        layers = []
        layers.append(Conv((5, 5, n_channel, 6))) # 32x32xn_channel -> 28x28x6
        
        if use_batch_normalization:
            layers.append(BatchNorm((28, 28, 6)))
            
        layers.append(Activation(tf.nn.relu))
        layers.append(Pooling()) # 28x28x6 -> 14x14x6
        layers.append(Conv((5, 5, 6, 16))) # 14x14x6 -> 10x10x16
        layers.append(Activation(tf.nn.relu))        
        layers.append(Pooling()) # 10x10x16 -> 5x5x16.
        layers.append(Flatten()) # 400
        layers.append(Dense(400, 120, tf.nn.relu)) # 400. Output = 120.
        layers.append(Dense(120, 84, tf.nn.relu)) # 400. Output = 120.
    
         # No activation needed in this layer as we are using logit with cross entropy
        layers.append(Dense(84, n_classes))
        
        self.layers = layers
        
    def fit(self, x):
        for layer in self.layers:
            x = layer.f_prop(x)
        
        return x      
    

## Training Pipeline
Create a training pipeline that uses the model to classify traffic signal data.

In [ ]:
import time

class Pipeline:
    def __init__(self, name, n_channels=1, learning_rate=0.0001, use_batch_normalization=False):
        self.name = name
        
        self.X = tf.placeholder(tf.float32, (None, 32, 32, n_channels))
        self.y = tf.placeholder(tf.int64, (None))
        
        self.logits = LeNet(n_channels, use_batch_normalization).fit(self.X)
        self.loss_operation = self._loss_op()
        self.training_operation = self._train_op(learning_rate)
        self.accuracy_operation = self._accuracy_op()
    
    def _loss_op(self):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y)
        
        return tf.reduce_mean(cross_entropy)
    
    def _train_op(self, learning_rate):        
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
        
        return optimizer.minimize(self.loss_operation)
    
    def _accuracy_op(self):                
        preds = tf.arg_max(self.logits, 1)
        correct_prediction = tf.equal(preds, self.y)
        
        return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def predict(self):
        return tf.arg_max(self.logits, 1)
    
    def evaluate(self, X_valid, y_valid, sess, batch_size=32):
        total_acc = 0
        total_loss = 0
        for offset in range(0, X_valid.shape[0], batch_size):
            end = offset + batch_size
            X_batch = X_valid[offset:end]
            y_batch = y_valid[offset:end]

            loss, acc = sess.run([self.loss_operation, self.accuracy_operation], feed_dict={self.X: X_batch, self.y: y_batch})
            total_loss += (loss * X_batch.shape[0])
            total_acc += (acc * X_batch.shape[0])

        return total_loss/X_valid.shape[0], total_acc/X_valid.shape[0]

    
    def train(self, X_train, y_train, X_valid, y_valid, X_test, y_test, epochs, batch_size=32, on_complete='eval'):
        max_accuracy = 0.0
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            
            if on_complete == 'save':
                saver = tf.train.Saver()

            num_examples = len(X_train)

            for i in range(epochs):
                X_train, y_train = shuffle(X_train, y_train)
                t0 = time.time()
                for offset in range(0, X_train.shape[0], batch_size):
                    end = offset + batch_size
                    sess.run(self.training_operation, feed_dict={
                        self.X: X_train[offset:end],
                        self.y: y_train[offset:end]})

                val_loss, val_acc = self.evaluate(X_valid, y_valid, sess)
                
                if val_acc > max_accuracy:
                    max_accuracy = val_acc
                    
                    if on_complete == 'save':
                        saver.save(sess, './{}'.format(self.name))                    

                print("Epoch {:2d} Time: {:.2f} s, Validation Loss={:.5f}, accuracy={:.1f} %".format(
                    i + 1,
                    (time.time() - t0), 
                    val_loss,
                    val_acc * 100
                ))
                
            if on_complete == 'eval':
                _, test_accuracy = self.evaluate(X_test, y_test, sess)
                print("Test Set Accuracy = {:.2f}%".format(test_accuracy * 100))
        
            if on_complete == 'save':
                print("Model w/ validation set accuracy: {:.1f} % saved at {}".format(max_accuracy*100.0, self.name))

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
def evaluate_test_set(model_name, X_test, y_test, pipeline):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver2 = tf.train.import_meta_graph('./{}.meta'.format(model_name))
        saver2.restore(sess, "./{}".format(model_name))

        _, test_accuracy = pipeline.evaluate(X_test, y_test, sess)
        print("Test Set Accuracy = {:.2f}%".format(test_accuracy * 100))
        
        predictions = sess.run([pipeline.predict()], feed_dict={pipeline.X: X_test, pipeline.y: y_test})
        
        return predictions

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
EPOCHS = 20
rate = 0.001

In [ ]:
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

tf.reset_default_graph()

final_model = "lenet-final"

print ("Train, validate and evaluate test set accuracy...")

pipeline = Pipeline(final_model, learning_rate=rate, use_batch_normalization=True)
pipeline.train(X_train_gray, y_train_augmented, X_valid_gray, y_valid, X_test_gray, y_test, epochs=EPOCHS, on_complete='save')

predictions = evaluate_test_set(final_model, X_test_gray, y_test, pipeline)

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.
import os
import matplotlib.image as mpimg
from skimage.transform import resize

def read_image(name):
    return mpimg.imread(name)

def resize_image(image, w=32, h=32):
    image_resized = resize(image, (w, h))
    return image_resized
    
# Read and save test images to array
original_test_images = [(i, read_image('web/' + i)) for i in os.listdir('web/')]
test_images = [(name, resize_image(image)) for name, image in original_test_images]

for name, image in test_images:
    skimage.io.imsave('./examples/' + name, image)    

In [ ]:
def show_new_images(images):
    fig, axs = plt.subplots(1, len(images), figsize=(10, 5))
    fig.subplots_adjust(hspace=0.1, wspace=0.1)
    axs = axs.ravel()

    for i in range(len(images)):           
        axs[i].imshow(images[i][1])
        axs[i].axis('off')

    plt.show()

In [ ]:
show_new_images(test_images)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.
    
my_labels = [18, 13, 25, 14, 12]
my_images_gray = np.sum(np.array(test_images)/3, axis=3, keepdims=True)

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

evaluate_test_set(final_model, my_images_gray, my_labels, pipeline)

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

softmax_logits = tf.nn.softmax(pipeline.logits)
top_k = tf.nn.top_k(softmax_logits, k=5)


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('./{}.meta'.format(final_model))
    saver.restore(sess, "./{}".format(final_model))
    my_softmax_logits = sess.run(softmax_logits, feed_dict={pipeline.X: my_images_gray})
    my_top_k = sess.run(top_k, feed_dict={pipeline.X: my_images_gray})

    n_cols = 6
    fig, axs = plt.subplots(len(test_images), n_cols, figsize=(15, 12))
    fig.subplots_adjust(hspace = .4, wspace=.2)
    axs = axs.ravel()

    for r, image in enumerate(test_images):
        c = r * n_cols
        
        axs[c].axis('off')
        axs[c].imshow(image)
        axs[c].set_title('Actual: {}'.format(my_labels[r]))
        
        for i, guess in zip(range(5), my_top_k[1][r][:5]): 
            index = np.argwhere(y_valid == guess)[0]
            
            color = 'g' if my_labels[r] == guess else 'r'
            
            pos = c + i + 1
            axs[pos].axis('off')
            axs[pos].imshow(X_valid[index].squeeze())
            axs[pos].set_title('Guess: {} ({:.0f}%)'.format(guess, 100 * my_top_k[0][r][i]), color=color)


### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be th8e same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")